In [1]:
# installs after TF setup https://www.tensorflow.org/install/pip 

# !pip install pandas
# !pip install mne
# !pip install scikit-learn

In [2]:
import os, glob, platform
import numpy as np
import numpy.matlib
import pickle
import pandas as pd
import pathlib
import matplotlib
import matplotlib.pyplot as plt
import mne
mne.__version__
from mne.viz import plot_alignment, snapshot_brain_montage
import shutil
from mne.datasets import eegbci
from sklearn.model_selection import train_test_split

# from mne_bids import write_raw_bids, BIDSPath, print_dir_tree, make_dataset_description
# from mne_bids.stats import count_events
import sys


In [3]:
path_utils = '/decoding_toolbox_py/helper_funcs' 
sys.path.append(path_utils)

In [4]:
''' VARIABLES '''

dataset = 'eeg'

amount_of_subjects = 1 # Change the range so the process is faster
if amount_of_subjects > 26: amount_of_subjects = 26
subjs_list = ['s{:02d}'.format(i) for i in range(1, amount_of_subjects+1) if i != 6 ] 
print(subjs_list)
nSubj = len(subjs_list)

numC = 8

angles = [i * 180./numC for i in range(numC)]

x_labels = np.array(angles)

resample = True # speeds up the procees but showing worse results overall
if resample: resample_frequency = 20 # in Hz, original freq is 500Hz

cfg_stim = dict()
cfg_stim['kappa'] = 4
cfg_stim['NumC'] = numC
cfg_stim['Tuning'] = 'vonmises'
# cfg_stim['Tuning'] = 'halfRectCos'
cfg_stim['offset'] = 0

cfg_train = dict()
cfg_train['gamma'] = 0.1
cfg_train['demean'] = True
cfg_train['returnPattern'] = True

cfg_test = dict()
cfg_test['demean'] = 'traindata'

['s01']


In [5]:
'''EEG Dataset'''
def read_data(
        number_of_repetition=3,
        resample=False,
        resample_frequency = 20,
        subjs_list = subjs_list,
        task = 'main'
        
        ):
    path = 'Cond_CJ_EEG'

    epochs = []
    all_epochs = []
    all_rawdata = []
    all_st_epochs = []
    all_st_rawdata = []
    for subject_id in subjs_list:
        preproc_path = os.path.join(path, subject_id)

        if task == 'main':
            epoch = mne.read_epochs(os.path.join(preproc_path, 'main_epo.fif'), verbose=False)
            
            if resample: 
                print('Frequency before:', epoch.info['sfreq'])
                epoch = epoch.resample(resample_frequency)
                print('Frequency after:' ,epoch.info['sfreq'])

            epochs.append(epoch.average())
            all_epochs.append(epoch)
            all_rawdata.append({'epoch_dat': epoch.get_data(), 'metadata': epoch.metadata})
            
        if task == 'stim':
        
            st_epoch = mne.read_epochs(os.path.join(preproc_path, 'mainstim_epo.fif'), verbose=False)
            # print(st_epoch.info['sfreq'])
            if resample: 
                print('Frequency before:', st_epoch.info['sfreq'])
                st_epoch = st_epoch.resample(resample_frequency)
                print('Frequency after:' ,st_epoch.info['sfreq'])
                
            all_st_epochs.append(st_epoch)
            all_st_rawdata.append(
                {
                'epoch_dat': st_epoch.get_data()[st_epoch.metadata['nrep'] == number_of_repetition,:,:] ,
                'metadata': st_epoch.metadata[st_epoch.metadata['nrep'] == number_of_repetition]
                }
                )
    if task == 'main':
        return all_rawdata
    else:
        return all_st_rawdata


In [6]:
all_rawdata = read_data (task = 'main', resample=False, resample_frequency=20)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [7]:
all_rawdata[0]['metadata'].columns

Index(['index', 'subj', 'nblock', 'ntrial', 'nrep', 'trial_type', 'cond-1',
       'cond', 'rDV', 'DV', 'resp', 'deci-2', 'deci-1', 'deci', 'corr-1',
       'r_map', 'correct', 'confi', 'RT', 'd1', 'conf_lvl', 'correct-1', 'd2',
       'd3', 'd4', 'd5', 'd6', 'o1', 'o2', 'o3', 'o4', 'o5', 'o6', 'confi-1',
       'conf_lvl-1'],
      dtype='object')

In [8]:
A = np.random.random((3,2,5))
print(A)
A = A.reshape((6,5))
print(A.shape)
A

[[[0.11679759 0.33998648 0.90650849 0.99838966 0.94697136]
  [0.8596475  0.9835257  0.96759688 0.21262496 0.23633638]]

 [[0.87280443 0.14283345 0.6272415  0.66526213 0.81408578]
  [0.87545364 0.98953588 0.09691274 0.66385269 0.10250404]]

 [[0.66690015 0.74896717 0.90500394 0.22545847 0.97563277]
  [0.72590262 0.4881958  0.46313171 0.32136119 0.57699   ]]]
(6, 5)


array([[0.11679759, 0.33998648, 0.90650849, 0.99838966, 0.94697136],
       [0.8596475 , 0.9835257 , 0.96759688, 0.21262496, 0.23633638],
       [0.87280443, 0.14283345, 0.6272415 , 0.66526213, 0.81408578],
       [0.87545364, 0.98953588, 0.09691274, 0.66385269, 0.10250404],
       [0.66690015, 0.74896717, 0.90500394, 0.22545847, 0.97563277],
       [0.72590262, 0.4881958 , 0.46313171, 0.32136119, 0.57699   ]])

In [9]:
b = np.random.random((5))
b = np.tile(b,3)
b

array([0.40911344, 0.81880415, 0.38111926, 0.94383717, 0.15539809,
       0.40911344, 0.81880415, 0.38111926, 0.94383717, 0.15539809,
       0.40911344, 0.81880415, 0.38111926, 0.94383717, 0.15539809])

In [10]:
neural_network = True
transpose = False
if neural_network:
    X = all_rawdata[0]['epoch_dat']
    y = all_rawdata[0]['metadata']['deci']
    for i in range(1, nSubj):
        X = np.concatenate((X, all_rawdata[i]['epoch_dat']), axis=0)
        y = np.concatenate((y, all_rawdata[i]['metadata']['deci']))
    print(X.shape, y.shape)
    if transpose: 
        X = np.einsum('ijk->ikj', X)

    print(X.shape, y.shape)
    numF, numC, numT = X.shape


(250, 32, 2876) (250,)
(250, 32, 2876) (250,)


In [15]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = keras.Sequential()
model.add(layers.LSTM(units=128, return_sequences=True, input_shape=(numC, numT))) 
model.add(layers.Dropout(0.2))
model.add(layers.LSTM(units=64))
# model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=1, activation='sigmoid'))  # Change units to 1 for binary classification and use sigmoid activation

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the EarlyStopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=1)

# Train the model with the EarlyStopping callback
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Epoch 1/50


2023-06-08 13:26:05.719209: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-08 13:26:05.720139: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-08 13:26:05.720648: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

13/13 [==============================] - ETA: 0s - loss: 0.6679 - accuracy: 0.6700

2023-06-08 13:26:07.818383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-08 13:26:07.819022: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-08 13:26:07.819619: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

13/13 [==============================] - 2s 66ms/step - loss: 0.6679 - accuracy: 0.6700 - val_loss: 0.6391 - val_accuracy: 0.6600
Epoch 2/50
13/13 [==============================] - 1s 39ms/step - loss: 0.6258 - accuracy: 0.6800 - val_loss: 0.6409 - val_accuracy: 0.6600
Epoch 3/50
13/13 [==============================] - 1s 40ms/step - loss: 0.6323 - accuracy: 0.6800 - val_loss: 0.6375 - val_accuracy: 0.6600
Epoch 4/50
13/13 [==============================] - 1s 44ms/step - loss: 0.6312 - accuracy: 0.6800 - val_loss: 0.6405 - val_accuracy: 0.6600
Epoch 5/50
13/13 [==============================] - 1s 43ms/step - loss: 0.6270 - accuracy: 0.6800 - val_loss: 0.6381 - val_accuracy: 0.6600
Epoch 6/50
13/13 [==============================] - 1s 43ms/step - loss: 0.6300 - accuracy: 0.6800 - val_loss: 0.6373 - val_accuracy: 0.6600
Epoch 7/50
13/13 [==============================] - 1s 44ms/step - loss: 0.6280 - accuracy: 0.6800 - val_loss: 0.6387 - val_accuracy: 0.6600
Epoch 8/50
13/13 [======

RF all subjects 80-20 = 0.556809024979855

RF, Logistic, SVC = 1 sub 0.66

XGB 1 sub 0.62

XGB 26 0.5495568090249798

Ada 0.6

GB 0.62

0.66


catboost and RF both on more data with stacked are perfoming the same

next im gonna add labels for the subjects


In [12]:
# X = all_rawdata[0]['epoch_dat']
# print(X.shape)
# # X = X.reshape(X.shape[0],-1)
# y = all_rawdata[0]['metadata']['deci']
# 
# X2 = all_rawdata[2]['epoch_dat' ]
# y2 = all_rawdata[2]['metadata']['deci']
# 
# X = np.concatenate((X,X2), axis = 0)
# y = np.concatenate((y,y2))
# 
# print(X.shape, y.shape)
# for i in range(1, nSubj):
# 
#     X_temp = all_rawdata[i]['epoch_dat']
#     X_temp = X_temp.reshape((X_temp.shape[0]*X_temp.shape[1], X_temp.shape[2]))
#     shapes[i+1] = shapes[i] + X_temp.shape[0]
#     y_temp = all_rawdata[i]['metadata']['deci']
#     y_temp = np.tile(y_temp, numC)
# 
#     X = np.vstack((X, X_temp))
#     y = np.concatenate((y, y_temp))
# 
# print(shapes)
# 
# print(X.shape, y.shape)
# if add_labels:
#     labels = np.zeros((nSubj, X.shape[0]))
#     for i in range (nSubj):
#         for j in range(shapes[i], shapes[i+1]):
#             labels[i,j] = 1
#     print(labels.shape)
#     
#     labels = np.transpose(labels)
#     X = np.hstack((X, labels))
#     print(X.shape, labels.shape)